In [1]:
# Dependencies
import requests
import pandas as pd
import matplotlib as plt
import numpy as np
import json
import sqlalchemy
from sqlalchemy import create_engine
from pymongo import MongoClient

In [ ]:
# Save url as variable
url = "https://us-states.p.rapidapi.com/all"

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "us-states.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers)
data = response.json()

# Print the response
print(response.text)

In [3]:
# Write the data to a json file
with open('data.json', 'w') as outfile:
    json.dump(data, outfile, indent=4)

In [ ]:
# Create a pandas data frame
df = pd.json_normalize(response.json())
df

In [ ]:
# Check data types
df.dtypes

In [6]:
# Check the counts
df.count()

name                          56
postal                        56
date                          56
nickname                      56
fips                          56
                              ..
symbols.heritage livestock     1
symbols.bat                    1
symbols.oyster                 1
symbols.cocktail               1
symbols.domestic animal        1
Length: 203, dtype: int64

In [7]:
# Slice the data frame to check for nulls
df.iloc[:,196:203]

,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop all territories
states_df = df.drop([2,11,36,41,47])
states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,symbols.handgun,symbols.hymn,symbols.star,symbols.astronomical symbol,symbols.emblem,symbols.heritage livestock,symbols.bat,symbols.oyster,symbols.cocktail,symbols.domestic animal
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Georgia,GA,"Jan 2, 1788",Peach State,13,Georgian,"[{'name': 'Atlanta', 'population': '504527'}, ...",Atlantic Ocean,Brasstown Bald,https://www.georgia.gov/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Drop half of the columns that have nulls
clean_states_df = states_df.drop(columns=['status','other_nicknames','standard_federal_region','status','other_nicknames','standard_federal_region','koppen_climate','population.density_km','elevation.min_ft','elevation.min_m','elevation.mean_ft','elevation.span_ft','elevation.mean_rank','elevation.span_m','elevation.mean_m','elevation.max_m','area.land_km','area.water_rank','area.total_rank','area.land_mi','area.water_km','area.total_km','area.land_percent','area.water_mi','area.water_percent','symbols.mascot','symbols.fruit','symbols.tree fruit','symbols.soil','symbols.game bird','symbols.reptile','symbols.amphibian','symbols.nut','symbols.vegetable','symbols.rock','symbols.saltwater fish','symbols.mammal','symbols.butterfly','symbols.cake','symbols.insect','symbols.mineral','symbols.spirit','symbols.fossil','symbols.freshwater fish','symbols.wildflower','symbols.shell','symbols.land mammal','symbols.fish','symbols.bolt action rifle','symbols.dog','symbols.marine mammal','symbols.sport','symbols.plant','symbols.metal','symbols.colors','symbols.drink','symbols.firearm','symbols.dinosaur','symbols.floral emblem','symbols.folk dance','symbols.grape','symbols.fruit and vegetable','symbols.musical instrument','symbols.primitive fish','symbols.grain','symbols.cooking pot','symbols.lichen','symbols.grass','symbols.marine reptile','symbols.dance','symbols.tartan','symbols.gold rush ghost town', 'symbols.marine animal','symbols.macroinvertebrate','symbols.wildlife animal','symbols.dessert','symbols.bug','symbols.beverage','symbols.herb','symbols.saltwater mammal','symbols.heritage cattle breed','symbols.pie','symbols.stone','symbols.tortoise','symbols.saltwater reptile','symbols.prepared food','symbols.waltz','symbols.cold water gamefish','symbols.crop','symbols.raptor','symbols.exercise','symbols.artifact','symbols.snackfood','symbols.prairie grass','symbols.slogan','symbols.river','symbols.rifle','symbols.red wine grape','symbols.white wine grape','symbols.flying fossil','symbols.gun', 'symbols.wild animal game species',	'symbols.sports car', 'symbols.crustacean',	'symbols.meat pie','symbols.berry',	'symbols.treat','symbols.ballad','symbols.cat','symbols.team sport','symbols.donut','symbols.bean',	'symbols.cookie','symbols.muffin','symbols.folk song', 'symbols.bee','symbols.mushroom', 'symbols.waterfowl','symbols.toy','symbols.tree nut','symbols.fruit tree','symbols.hockey team','symbols.football team','symbols.endangered species','symbols.soft drink','symbols.poultry','symbols.saltwater game fish','symbols.freshwater game fish','symbols.spider','symbols.cryptid','symbols.bush','symbols.freshwater trout','symbols.salamander', 'symbols.marsupial','symbols.flying mammal','symbols.game animal','symbols.microbe','symbols.coral','symbols.appetizer','symbols.heritage horse','symbols.jewelry','symbols.bread','symbols.shrub','symbols.pastry','symbols.dish','symbols.handgun','symbols.hymn', 'symbols.astronomical symbol','symbols.emblem','symbols.heritage livestock','symbols.bat','symbols.oyster','symbols.cocktail','symbols.domestic animal','symbols.hero','symbols.shellfish',"symbols.children's flower",'symbols.heroine','symbols.polka','symbols.music','symbols.game mammal','symbols.aquatic animal','symbols.frog','symbols.star','symbols.horse','symbols.neckwear','symbols.silver rush ghost town','symbols.marine fish','symbols.fabric',	'symbols.pets',	'symbols.cactus','symbols.animal','population.density_mi','ap_abbreviation','gpo_abbreviation','symbols.flower','symbols.song','symbols.gem','subdivisions','census_bureau.region','census_bureau.division','symbols.bird'])
clean_states_df

,name,postal,date,nickname,fips,demonym,cities,lowest_point,highest_point,website,...,motto,capital.name,capital.latitude,capital.longitude,population.total,elevation.max_rank,elevation.max_ft,area.total_mi,area.land_rank,symbols.tree
0,Alabama,AL,"Dec 14, 1819",Heart of Dixie,01,Alabamian,"[{'name': 'Huntsville', 'population': '215006'...",Gulf of Mexico,Cheaha Mountain,http://www.alabama.gov,...,Audemus jura nostra defendere,Montgomery,32.377716,-86.300568,5024279,38,2413,52420.07,28,Southern Longleaf Pine
1,Alaska,AK,"Jan 3, 1959",The Last Frontier,02,Alaskan,"[{'name': 'Anchorage', 'population': '291538'}...","Gulf of Alaska, Bering Sea, and Arctic Ocean",Denali,http://alaska.gov/,...,North to the Future,Juneau,58.301598,-134.420212,733391,1,20310,665384.04,1,Sitka Spruce
3,Arizona,AZ,"Feb 14, 1912",Grand Canyon State,04,Arizonan,"[{'name': 'Phoenix', 'population': '1660272'},...","Colorado River at Sonora border, San Luis, Ari...",Humphreys Peak,https://az.gov/,...,Ditat Deus,Phoenix,33.448143,-112.096962,7151502,12,12637,113990.3,6,Palo Verde
4,Arkansas,AR,"Jun 15, 1836",Natural State,05,Arkansan,"[{'name': 'Little Rock', 'population': '197881...",Ouachita River at Louisiana border,Mount Magazine,http://www.arkansas.gov/,...,Regnat populus,Little Rock,34.746613,-92.288986,3011524,37,2753,53178.55,27,Pine Tree
5,California,CA,"Sep 9, 1850",Golden State,06,Californian,"[{'name': 'Los Angeles', 'population': '399798...",Badwater Basin in Death Valley,Mount Whitney,https://www.ca.gov/,...,Eureka,Sacramento,38.576668,-121.493629,39538223,2,14505,163694.74,3,"Coast Redwood,Giant Sequoia"
6,Colorado,CO,"Aug 1, 1876",Centennial State,08,Coloradan,"[{'name': 'Denver', 'population': '719238'}, {...",Arikaree River at Kansas border,Mount Elbert,http://www.colorado.gov/,...,Nil sine numine,Denver,39.739227,-104.984856,5773714,3,14440,104093.67,8,Colorado Blue Spruce
7,Connecticut,CT,"Jan 9, 1788",Constitution State,09,Connecticuter,"[{'name': 'Bridgeport', 'population': '144900'...",Long Island Sound,Massachusetts border on the southern slope of ...,https://portal.ct.gov/,...,Qui transtulit sustinet,Hartford,41.764046,-72.682198,3605944,39,2379,5543.41,48,Charter Oak
8,Delaware,DE,"Dec 7, 1787",First State,10,Delawarean,"[{'name': 'Wilmington', 'population': '70635'}...",Atlantic Ocean,Near the Ebright Azimuth,https://delaware.gov/,...,Liberty and independence,Dover,39.157307,-75.519722,989948,54,447,2488.72,50,American Holly
9,Florida,FL,"Mar 3, 1845",Sunshine State,12,Floridian,"[{'name': 'Jacksonville', 'population': '90795...",Atlantic Ocean and Gulf of Mexico,Britton Hill,http://myflorida.com/,...,In God we trust,Tallahassee,21.307442,-157.857376,21538187,56,345,65757.7,26,Sabal Palm
10,Georgia,GA,"Jan 2, 1788",Peach State,13,Georgian,"[{'name': 'Atlanta', 'population': '504527'}, ...",Atlantic Ocean,Brasstown Bald,https://www.georgia.gov/,...,"Wisdom, Justice, Moderation",Atlanta,30.438118,-84.281296,10711908,25,4784,59425.15,21,Southern Live Oak


In [ ]:
# Show all columns of data frame
pd.options.display.max_columns = None
clean_states_df = clean_states_df.replace({np.nan: None})
clean_states_df.to_html('states.html',index=False,classes=["table","table-striped","table-bordered"])
clean_states_df

In [ ]:
# Save url as variable
url = "https://jsearch.p.rapidapi.com/search"

# Set querystring 
querystring = {"query":"Entry Level Data Scientist in USA","num_pages":"20"}

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_scientists = response.json()

# Write the data to a json file
with open('entry_level_data_scientists.json', 'w') as outfile:
    json.dump(entry_level_data_scientists, outfile, indent=4)


# Print response
print(response.text)

In [ ]:
# Save url as variable
url = "https://jsearch.p.rapidapi.com/search"

# Set querystring
querystring = {"query":"Entry Level Data Analyst in USA","num_pages":"20"}

# Set headers
headers = {
	"X-RapidAPI-Key": "575ca32490msh5a85e1221a9daf0p10835bjsnfdf17e56a208",
	"X-RapidAPI-Host": "jsearch.p.rapidapi.com"
}

# Make a GET request and save in variable
response = requests.request("GET", url, headers=headers, params=querystring)
entry_level_data_analyst = response.json()

# Write the data to a json file
with open('entry_level_data_analyst.json', 'w') as outfile:
    json.dump(entry_level_data_analyst, outfile, indent=4)


# Print response
print(response.text)

In [ ]:
# Select certain columns to be used and turn json file into a pandas data frame 
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df = pd.json_normalize(entry_level_data_analyst, 'data')
new_df = df.loc[:, columns]
new_df

In [ ]:
# Select certain columns to be used and turn json file into a pandas data frame 
columns = ["employer_name","job_title","job_apply_link","job_description","job_is_remote","job_city","job_state","job_country","job_latitude","job_longitude"]
df2 = pd.json_normalize(entry_level_data_scientists, 'data')
new_data_df = df2.loc[:, columns]
new_data_df

In [ ]:
# Add the two data frames together 
result_df = pd.concat([new_df, new_data_df], axis=0)
result_df = result_df.replace({np.nan: None})
result_df = result_df.replace({'\n': ' '}, regex=True)
result_df.to_html('jobs_table.html',index=False,classes=["table","table-striped","table-bordered"])
result_df

In [16]:
# Connect to the MongoDB server
client = MongoClient()

# Choose the database you want to use
db = client.data

# Insert the clean_states_df data frame into a collection named "clean_states"
db.states.insert_many(clean_states_df.to_dict('records'))

# Insert the result_df data frame into a collection named "result"
db.jobs.insert_many(result_df.to_dict('records'))